# packet

> TODO fill in description

In [ ]:
#| default_exp packet

In [ ]:
#| hide
from nbdev.showdoc import *; 

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#|export
from __future__ import annotations
import asyncio
from typing import Any, Union, List, Hashable, Dict, Set, Tuple
from enum import Enum
from dataclasses import dataclass, field
from datetime import datetime

import fbdev
from fbdev.utils import AddressableMixin

In [ ]:
#|hide
show_doc(fbdev.packet.NullPayload)

---

### NullPayload

>      NullPayload ()

*Initialize self.  See help(type(self)) for accurate signature.*

In [ ]:
#|export
class NullPayload:
    _instance = None
    def __new__(cls):
        if cls._instance is None:
            cls._instance = super(NullPayload, cls).__new__(cls)
        return cls._instance
    def __repr__(self):
        return "<NullPayload>"

In [ ]:
#|hide
show_doc(fbdev.packet.Packet)

---

### Packet

>      Packet (_data:Any, _id=None)

*Initialize self.  See help(type(self)) for accurate signature.*

In [ ]:
#|export
class Packet:
    def __init__(self, *, _data:Any, _id=None):
        self._id:Hashable = _id
        self._data:Any = _data
        self._dtype:type = type(_data)
            
    @property
    def id(self): return self._id
    @property
    def is_empty(self): return self._dtype == NullPayload
    @property
    def dtype(self): return self._dtype
    
    def _move(self, address):
        pass
    
    def get_deep_copy(self):
        pass

In [ ]:
#|hide
show_doc(fbdev.packet.PacketActivity)

---

### PacketActivity

>      PacketActivity (packet:fbdev.packet.Packet,
>                      timestamp:datetime.datetime=None)

*Initialize self.  See help(type(self)) for accurate signature.*

In [ ]:
#|export
class PacketActivity:
    def __init__(self, packet:Packet, timestamp: datetime = None):
        self.packet_id:Hashable = packet.id
        if timestamp is None: timestamp = datetime.now()  # TODO not timezone aware
        self.timestamp:datetime = timestamp

In [ ]:
#|hide
show_doc(fbdev.packet.PacketCreation)

---

### PacketCreation

>      PacketCreation (packet:fbdev.packet.Packet, address,
>                      timestamp:datetime.datetime=None)

*Initialize self.  See help(type(self)) for accurate signature.*

In [ ]:
#|export
class PacketCreation(PacketActivity):
    def __init__(self, packet:Packet, address, timestamp: datetime = None):
        super().__init__(packet, timestamp)
        self.packet_dtype:type = packet.dtype
        self.address:Tuple = address

In [ ]:
#|hide
show_doc(fbdev.packet.PacketMovement)

---

### PacketMovement

>      PacketMovement (packet:fbdev.packet.Packet, origin:Any, dest:Any,
>                      port_type:fbdev.port.PortType, port_name:str,
>                      timestamp:datetime.datetime=None)

*Initialize self.  See help(type(self)) for accurate signature.*

In [ ]:
#|export
class PacketMovement(PacketActivity):
    def __init__(self, packet:Packet, origin: Any, dest: Any, port_type: fbdev.port.PortType, port_name: str, timestamp: datetime = None):
        super().__init__(packet, timestamp)
        self.origin:Tuple = origin
        self.dest:Tuple = dest
        self.port_type:fbdev.port.PortType = port_type
        self.port_name:str = port_name

In [ ]:
#|hide
show_doc(fbdev.packet.PacketRegistry)

---

### PacketRegistry

>      PacketRegistry ()

*Initialize self.  See help(type(self)) for accurate signature.*

In [ ]:
#|export
class PacketRegistry:
    def __init__(self): 
        self._packets:Dict[Hashable, Packet] = {}
        self._num_packets = 0
        self._consumed_packets:Set[Hashable] = set()
        self._locations:Dict[Packet, Tuple] = {}
        self._history: List[PacketActivity] = []
        
    def get_location(self, packet:Packet):
        if not self.is_registered(packet): raise ValueError("Packet not in registry.")
        return self._locations[packet]
    
    def is_consumed(self, packet:Packet):
        return packet.id in self._consumed_packets
    
    def is_registered(self, packet:Packet):
        return packet.id in self._packets
        
    def create(self, data:Any, address):
        if type(data) == Packet:
            raise ValueError("Cannot create packet from another packet.")
        packet = Packet(_data=data, _id=self._num_packets)
        self._num_packets += 1
        self._packets[packet.id] = packet
        self._locations[packet] = address
        self._history.append(PacketCreation(packet, address))
        return packet
    
    def create_empty(self, address):
        return self.create(NullPayload(), address)
        
    def register_move(self, packet:Packet, dest:AddressableMixin, port_type:fbdev.port.PortType, port_name:str):
        if not self.is_registered(packet): raise ValueError("Packet not in registry.")
        if self.is_consumed(packet): raise ValueError("Packet is consumed.")
        current_address = self.get_location(packet)
        if dest.address == current_address: raise ValueError("Packet is already at destination.")
        dest_address = dest.address
        moving_via = (port_type, port_name)
        self._history.append(PacketMovement(packet, current_address, dest_address, port_type, port_name))
        self._locations[packet] = dest_address
        
    async def consume(self, packet:Packet):
        if not self.is_registered(packet): raise ValueError("Packet not in registry.")
        if self.is_consumed(packet): raise ValueError("Packet already consumed.")
        del self._locations[packet]
        self._consumed_packets.add(packet.id)
        data = packet._data # TODO this will be changed when we also have remote packets
        return data
    
    async def peek(self, packet:Packet):
        data = self._data # TODO this will be changed when we also have remote packets
        return data

In [ ]:
#|hide
show_doc(fbdev.packet.PacketHandler)

---

### PacketHandler

>      PacketHandler
>                     (registry:Union[fbdev.packet.PacketRegistry,fbdev.packet.P
>                     acketHandler], parent_node:fbdev.node.Node)

*A handler for packets for Nodes. The handler knows its own address, and verifies the packets against it.*

In [ ]:
#|export
class PacketHandler:
    """
    A handler for packets for Nodes. The handler knows its own address, and verifies the packets against it.
    """
    def __init__(self, registry:PacketRegistry, parent_node:fbdev.node.Node):
        self._registry:PacketRegistry = registry
        self._parent_node:fbdev.node.Node = parent_node
        
    def _verify_location(self, packet:Packet):
        if self._registry.get_location(packet) != self._parent_node.address:
            raise ValueError(f"Packet location is {self._registry.get_location(packet)}, but expected {self._parent_node.address}.")
        
    def is_consumed(self, packet:Packet):
        return self._registry.is_consumed(packet)
        
    def create(self, data:Any):
        return self._registry.create(data, self._parent_node.address)
    
    def create_empty(self):
        return self._registry.create_empty()
    
    def register_move(self, packet:Packet, dest:AddressableMixin, port_type:fbdev.port.PortType, port_name:str):
        if dest.address != self._parent_node.address:
            self._verify_location(packet) # If the packet is being sent away, then verify that it is currently located at self._parent_node
        self._registry.register_move(packet, dest, port_type, port_name)
        
    async def consume(self, packet:Packet):
        self._verify_location(packet)
        return await self._registry.consume(packet)
    
    async def peek(self, packet:Packet):
        self._verify_location(packet)
        return await self._registry.peek(packet)
    
    def get_location(self, packet:Packet):
        return self._registry.get_location(packet)